In [2]:
import os
from dotenv import load_dotenv, dotenv_values
from pathlib import Path
from yahoo_oauth import OAuth2
import json
import pandas as pd

In [3]:
load_dotenv()

True

In [4]:
df = pd.DataFrame(columns=[
    'player_key', 'player_name', 'team',
    'pg', 'sg', 'sf', 'pf', 'c',
    'owner_team_key', 'owner_team_name'
])

oauth = OAuth2(None, None, from_file=f'{os.getenv('FILES_DIR')}/private.json')
if not oauth.token_is_valid():
    oauth.refresh_access_token()

external_dotenv = Path(f'{os.getenv('FILES_DIR')}/.env')
load_dotenv(dotenv_path=external_dotenv)

GAME_KEY = os.getenv('GAME_KEY')
LEAGUE_ID = os.getenv('LEAGUE_ID')

page_num = 0
count = 0
while True:
    uri = f'players;start={page_num};status=T/ownership'
    url = 'https://fantasysports.yahooapis.com/fantasy/v2/league/'+GAME_KEY+'.l.'+LEAGUE_ID+'/'+uri
    response = oauth.session.get(url, params={'format':'json'})
    data = json.loads(response.text)

    try:
        data['fantasy_content']['league'][1]['players']['0']['player'][0][0]
    except TypeError:
        break

    for each1 in range(24):
        for each2 in range(13):
            try:
                key_value = data['fantasy_content']['league'][1]['players'][str(each1)]['player'][0][each2]
            except KeyError:
                continue
            try:
                key_lookup = list(key_value.keys())[0]
            except AttributeError:
                continue
            value_lookup = list(key_value.values())[0]
            if key_lookup == 'player_key':
                df.loc[count,'player_key'] = value_lookup
            elif key_lookup == 'name':
                df.loc[count,'player_name'] = value_lookup['full']
            elif key_lookup == 'editorial_team_abbr':
                df.loc[count,'team'] = value_lookup
            elif key_lookup == 'display_position':
                list_lookup = value_lookup.split(',')
                df.loc[count,'pg'] = 'PG' in list_lookup
                df.loc[count,'sg'] = 'SG' in list_lookup
                df.loc[count,'sf'] = 'SF' in list_lookup
                df.loc[count,'pf'] = 'PF' in list_lookup
                df.loc[count,'c'] = 'C' in list_lookup
            key_value = data['fantasy_content']['league'][1]['players'][str(each1)]['player'][1]['ownership']
            df.loc[count,'owner_team_key'] = key_value['owner_team_key']
            df.loc[count,'owner_team_name'] = key_value['owner_team_name']
        count += 1

    page_num += each1 + 1

[2023-11-20 23:18:34,228 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2023-11-20 23:18:34,228 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 704.8138003349304
[2023-11-20 23:18:34,229 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2023-11-20 23:18:34,230 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 704.8158438205719
[2023-11-20 23:18:34,231 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [5]:
load_dotenv()
df.to_csv(f'{os.getenv('FILES_DIR')}/yahoo.csv')